In [ ]:
import pandas as pd
import numpy as np
!pip install stanza
import stanza
from sklearn.feature_extraction.text import TfidfVectorizer
import re
!pip install bs4
from bs4 import BeautifulSoup
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct
!pip install contractions
import contractions
#!pip uninstall transformersy
!pip install --no-cache-dir transformers sentencepiece
from transformers import pipeline
import re
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import itertools
import copy
from transformers import AutoTokenizer, AutoModelForMaskedLM
!pip install -U sentence-transformers
!pip install detoxify
!pip install --upgrade git+https://github.com/flairNLP/flair.git
import pandas as pd
from detoxify import Detoxify
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from flair.embeddings import FlairEmbeddings
!pip install mudes
from mudes.app.mudes_app import MUDESApp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-bwgk84hu
  Running command git clone -q

Read Bad words from Stanford Dataset

In [ ]:
with open('bad_word.txt') as f:
    bad_words =[line.rstrip("\n") for line in f.readlines()]

Reading the Reddit Dataset

In [ ]:
df = pd.read_csv('labeled_comments.csv',  error_bad_lines=False)
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,comments,toxicity_score,toxicity_classification
0,I have never met such a dumb fucking person li...,0.8000,1
1,You are a bloody asshole leader and I hate you.,0.8000,1
2,I have never met such a stupid and foolish per...,0.0002,0
3,You are a bitch. I do not like you.,0.8000,1
4,Never in my life have a met such a foolish per...,0.0020,0


Make Our Labels based on the toxicity score

In [ ]:
label =[]
for i in range(len(df)):
  if df.toxicity_score[i]>0.5:
    label.append(1)
  else:
    label.append(0)
df['label'] = label
df.head()

,comments,toxicity_score,toxicity_classification,label
0,I have never met such a dumb fucking person li...,0.8000,1,1
1,You are a bloody asshole leader and I hate you.,0.8000,1,1
2,I have never met such a stupid and foolish per...,0.0002,0,0
3,You are a bitch. I do not like you.,0.8000,1,1
4,Never in my life have a met such a foolish per...,0.0020,0,0


In [ ]:
# get only sentence 1 for good sentences
for i in range(len(df)):
  if df.label[i]==0:
    df['comments'][i] = df['comments'][i].split(".")[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
def text_prepare(text):
    #removing urls
    text = re.sub(r'http\S+', '', text)
    text = BeautifulSoup(text).get_text()
    #removing emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = contractions.fix(text)
    #removing email
    text =re.sub(r'[A-Za-z0-9]@[A-Za-z]\.?[A-Za-z0-9]*', "", text)
    #Keeping only alphabets
    regex = re.compile('[^a-zA-Z ]')
    text = regex.sub(' ',str(text))
    text = re.sub(' +', ' ', text)
    text = text.lower()
    # delete stopwords from text
    # text = ' '.join([word for word in text.split()]) 
    # text = text.strip()
    return text

In [ ]:
# Data cleaning
df['comments']= df['comments'].apply(lambda x:text_prepare(x))

In [ ]:
# getting good sentences
good_sentences = df[df.label==0][:2500].reset_index().drop(['index', 'toxicity_classification', 'toxicity_score', 'label'], axis=1)

Retrive- POS Tagging, searching

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!


In [ ]:
app = MUDESApp("en-large", use_cuda=False)
def replace_BW(sent):
  spans = app.predict_toxic_spans(sent, spans=True)
  for i in spans[::-1]:
    sent = sent[:i[0]]+ 'BW' +sent[i[1]+1:]
  return sent

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

In [ ]:
def retrive_pos_tagging_good(balanced_dataset):
  pos = []
  pos_dict = []
  for i in range(len(balanced_dataset)):
    doc = nlp(balanced_dataset.comments[i])
    temp = []
    temp_dict = {}
    for sent in doc.sentences:
      for word in sent.words:
        temp.append(word.xpos)
        if word.xpos in temp_dict:
          temp_dict[word.xpos].append(word.text)
        else:
          temp_dict[word.xpos] = [word.text]
    pos.append(temp)
    pos_dict.append(temp_dict)
  return pos, pos_dict

In [ ]:
def retrive_pos_tagging_bad(balanced_dataset):
  temp_comments = []
  for i in range(len(balanced_dataset)):
    sent = balanced_dataset.comments[i]
    sent = replace_BW(sent)
    temp_comments.append(sent)

  balanced_dataset['new_comments'] = temp_comments
  pos = []
  pos_dict = []
  for i in range(len(balanced_dataset)):
    doc = nlp(balanced_dataset.new_comments[i])
    temp = []
    temp_dict = {}
    for sent in doc.sentences:
      for word in sent.words:
        if word.text == 'BW':
          temp.append('BW')
        else:
          temp.append(word.xpos)
          if word.xpos in temp_dict:
            temp_dict[word.xpos].append(word.text)
          else:
            temp_dict[word.xpos] = [word.text]
    pos.append(temp)
    pos_dict.append(temp_dict)
  return pos, pos_dict

TFIDF Search

In [ ]:
pos, pos_dict = retrive_pos_tagging_good(good_sentences)
good_sentences['pos'] = pos
good_sentences['pos'] = good_sentences['pos'].str.join(" ")
good_sentences['pos_dict'] = pos_dict

In [ ]:
good_sentences.pos[0]

'PRP VBP RB VBN PDT DT JJ CC JJ NN IN PRP RB'

In [ ]:
def get_tfidf_vectorizer(good_sentences, bad_sentences):
  balanced_dataset = pd.concat([good_sentences, bad_sentences], axis=0, ignore_index=True)
  tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.9, min_df=5, token_pattern='(\S+)')
  tf_idf_matrix = tfidf_vectorizer.fit(balanced_dataset['pos'])
  tf_idf_matrix_good = tfidf_vectorizer.transform(good_sentences['pos'])
  return tfidf_vectorizer, tf_idf_matrix_good

In [ ]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)
    matches = csr_matrix((data,indices,indptr),shape=(M,N)).toarray()
    ans =[]
    for i in range(len(matches)):
      a = matches[i]
      ans.append(np.argwhere(a>0))
    return ans

In [ ]:
def find_similar_good_sentences(bad_sentence, good_sentences, ans):
  similar_df= {'bad':[], 'good':[], 'bad_pos':[], 'good_pos':[], 'bad_pos_map':[], 'good_pos_map':[]}  
  similar_df['bad'].append(bad_sentence.comments[0])
  similar_df['bad_pos'].append(bad_sentence.pos[0])
  similar_df['bad_pos_map'].append(bad_sentence.pos_dict[0])
  similar_good = []
  similar_pos =[]
  similar_pos_map = []
  for i in ans:
    similar_good.append(good_sentences.comments[i[0]])
    similar_pos.append(good_sentences.pos.iloc[i[0]])
    similar_pos_map.append(good_sentences.pos_dict[i[0]])
  similar_df['good'].append(similar_good)
  similar_df['good_pos'].append(similar_pos)
  similar_df['good_pos_map'].append(similar_pos_map)
  return similar_df

Generate

In [ ]:
def make_assignments(list_1,list_2):
  unique_combinations = []
  # permut = itertools.permutations(list_1, len(list_2))
  # print(permut)

  if len(list_1)<len(list_2):
      permut = itertools.permutations(list_2, len(list_1))
      for comb in permut:
        zipped = zip(list_1,comb)
        unique_combinations.append(list(zipped))

  else:
    permut = itertools.permutations(list_1, len(list_2))
    for comb in permut:
        zipped = zip(comb,list_2)
        unique_combinations.append(list(zipped))
  
  return unique_combinations  
def generate(xi, pi, pi_dash, bad_words, pi_map, pi_dash_map):
  ti = set(pi)
  ti_dash = set(pi_dash)
  t_shared = ti.intersection(ti_dash)
  c0 = ["<mask>"]*len(pi_dash)
  ci = [c0]
  for inx, tk in enumerate(t_shared):
    if tk=='BW':
      continue
    wk = set(pi_map[tk])
    sk = [i for i, x in enumerate(pi_dash) if x == tk]
    assignments = make_assignments(wk, sk)
    if len(assignments)==0:
      continue
    new_ci = []
    for assignment in assignments:
      for prev_sent in ci:
        for word, indx in assignment:
          temp = copy.deepcopy(prev_sent)
          temp[indx]= word
        new_ci.append(temp)
    ci = new_ci
  return ci

In [ ]:
def fill_mask_word_roberta(input_string, classifier_roberta, puncs):
    if input_string.split(" ").count('<mask>') ==0:
       return input_string.split("[SEP]",1)[1].strip() 
    result = classifier_roberta(input_string)
    ans = input_string
    if ans.split(" ").count('<mask>') ==1:
       result = [result]
    for each_mask in result:
        for i in range(len(each_mask)):
            filler_word = each_mask[i]['token_str'].strip()
            if filler_word.lower() not in bad_words and filler_word not in puncs:
                ans = re.sub('<mask>', filler_word, ans, count=1)
                break
            if i == len(each_mask) - 1:
                ans = re.sub('<mask>', '', ans, count=1)
                break 
    return ans.split("[SEP]",1)[1].strip()


In [ ]:
def remove_adjacent(seq): # works on any sequence, not just on numbers
  i = 1
  n = len(seq)
  while i < n: # avoid calling len(seq) each time around
    if seq[i] == seq[i-1]:
      del seq[i]
      # value returned by seq.pop(i) is ignored; slower than del seq[i]
      n -= 1
    else:
      i += 1

In [ ]:
def embeddings_cosine_toxicity_perplexity(toxic_model, embedding_model, perplex_model, original, created_list):
  
  # Here we shall first make the embeddings of the sentences
  original_sent_embeddings = embedding_model.encode(original)
  created_sent_embeddings = embedding_model.encode(created_list)
  my_selection = {"original_sentence":original,"generated_sentences":created_list, "cosine_score":[], "non-toxicity_score":[], "perplexity_score":[]}
  cosine_calc = []
  toxic_calc = []
  perplex_calc = []

  # Now we will calclate the cosine with the original sentences
  for indx, i in enumerate(created_sent_embeddings):
    toxic_calc.append(1-(toxic_model.predict(created_list[indx])['toxicity']))
    perplex_calc.append(perplex_model.calculate_perplexity(created_list[indx])*-1)
    cosine_calc.append(cosine_similarity(original_sent_embeddings.reshape(1,-1), i.reshape(1,-1)).astype(str)[0][0])
  my_selection.update({"cosine_score": cosine_calc})
  my_selection.update({"non-toxicity_score":toxic_calc})
  my_selection.update({"perplexity_score":perplex_calc})
  
  return my_selection

In [ ]:

# tokenizer = AutoTokenizer.from_pretrained("conjuring92/distilroberta-base-finetuned-toxic")

model = AutoModelForMaskedLM.from_pretrained("conjuring92/distilroberta-base-finetuned-toxic")

# def get_prediction (sent):
    
#     token_ids = tokenizer.encode(sent, return_tensors='pt')
#     masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
#     masked_pos = [mask.item() for mask in masked_position ]

#     with torch.no_grad():
#         output = model(token_ids)

#     last_hidden_state = output[0].squeeze()

#     list_of_list =[]
#     for index,mask_index in enumerate(masked_pos):
#         mask_hidden_state = last_hidden_state[mask_index]
#         idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
#         words = [tokenizer.decode(i.item()).strip() for i in idx]
#         list_of_list.append(words)
#         print ("Mask ",index+1,"Guesses : ",words)
    
#     best_guess = ""
#     for j in list_of_list:
#         best_guess = best_guess+" "+j[0]
        
#     return best_guess

# sentence = " ".join(ci[3])
# print ("Original Sentence replaced with mask: ",sentence)
# print ("\n")

# predicted_blanks = get_prediction(sentence)
# print ("\nBest guess for fill in the blank :::",predicted_blanks)

Original Sentence replaced with mask:  <mask> <mask> <mask> and you hate <mask> <mask> a bloody leader


Mask  1 Guesses :  ['You', 'you', 'I', '', 'your']
Mask  2 Guesses :  ['hate', 'a', 'are', 'love', 'your']
Mask  3 Guesses :  [',', 'hate', 'you', 'it', 'wrong']
Mask  4 Guesses :  ['to', 'you', 'it', 'America', 'him']
Mask  5 Guesses :  ['be', 'have', 'is', 'as', 'with']

Best guess for fill in the blank :::  You hate , to be


In [ ]:
def get_civil_sentence(bad_sentence, good_sentences):
  bad_sentence = text_prepare(bad_sentence)
  bs = {'comments': bad_sentence}
  bs = pd.DataFrame([bs])
  # RETRIVE
  pos, pos_dict = retrive_pos_tagging_bad(bs)
  bs['pos'] = ' '.join(pos[0])
  bs['pos_dict'] = pos_dict
  print(bs.comments[0], pos)
  tfidf_vectorizer, tf_idf_matrix_good = get_tfidf_vectorizer(good_sentences, bs) 
  tf_idf_matrix_bad = tfidf_vectorizer.transform(bs['pos'])
  ans = awesome_cossim_top(tf_idf_matrix_bad, tf_idf_matrix_good.transpose(), 10, 0.02)
  similar_df = pd.DataFrame(find_similar_good_sentences(bs, good_sentences, ans[0]))
  # GENERATE
  ci = []
  for j in range(0,10):
    b = len(similar_df.bad[0].split(" "))
    g = len(similar_df.good[0][j].split(" "))
    if g<=b+5 and g>=b-5:
      ci+=generate(similar_df.bad[0], similar_df.bad_pos[0].split(), similar_df.good_pos[0][j].split(), bad_words, similar_df.bad_pos_map[0], similar_df.good_pos_map[0][j])
  print(ci)
  # fill masks
  classifier_roberta = pipeline('fill-mask', model="conjuring92/distilroberta-base-finetuned-toxic")
  puncs = ['.', ',']
  civil_sentences= []
  for c in ci:
    out = fill_mask_word_roberta(similar_df.bad[0]+"[SEP] "+" ".join(c), classifier_roberta, puncs).split(" ")
    remove_adjacent(out)
    civil_sentences.append(" ".join(out))
  # SELECTION
  toxic_model = Detoxify('original')
  embedding_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
  perplex_model = FlairEmbeddings('news-forward').lm
  original_sent = similar_df.bad[0]
  created_sent = civil_sentences

  # Running my func
  my_caclulations = embeddings_cosine_toxicity_perplexity(toxic_model, embedding_model, perplex_model, original_sent, created_sent)

  # Printing
  my_calculations_df = pd.DataFrame.from_dict(my_caclulations)
  my_calculations_df["sum_factor"] = my_calculations_df.sum(axis=1)
  return my_calculations_df


ci = get_civil_sentence('What the fuck are you doing you morons!', good_sentences)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

what the fuck are you doing you morons  [['WP', 'DT', 'NN', 'VBP', 'PRP', 'VBG', 'PRP', 'BW']]
[['<mask>', 'are', 'you', 'doing', '<mask>', 'the', '<mask>'], ['<mask>', 'fuck', 'doing', 'you', '<mask>', '<mask>'], ['<mask>', '<mask>', 'doing', 'you', '<mask>', 'fuck'], ['<mask>', 'fuck', '<mask>', 'you', 'doing', '<mask>'], ['<mask>', '<mask>', '<mask>', 'you', 'doing', 'fuck'], ['what', 'the', '<mask>', 'fuck']]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [ ]:
ci

,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
0,what the fuck are you doing you morons,what are you doing in the world,0.61665195,0.997477,-2.282938,-1.285461
1,what the fuck are you doing you morons,what fuck doing you mor ons,0.90264314,0.002332,-11.601321,-11.598988
2,what the fuck are you doing you morons,what are doing you the fuck,0.90458417,0.004926,-5.689642,-5.684717
3,what the fuck are you doing you morons,what fuck are you doing ?,0.88920575,0.008271,-4.635203,-4.626932
4,what the fuck are you doing you morons,what are you doing fuck,0.91510415,0.006622,-4.871469,-4.864847
5,what the fuck are you doing you morons,what the fuck,0.8376485,0.009337,-6.729740,-6.720403


In [ ]:
ci[ci['sum_factor'] == ci['sum_factor'].max()]

,original_sentence,generated_sentences,cosine_score,non-toxicity_score,perplexity_score,sum_factor
0,what the fuck are you doing you morons,what are you doing in the world,0.61665195,0.997477,-2.282938,-1.285461
